# Capstone Study

In [1]:
import utils
import plots
import classifiers as clfs

%matplotlib inline

/media/rodsnjr/Files/miniconda3/envs/udacity-ml/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn import model_selection
from sklearn.metrics import roc_auc_score, accuracy_score

def test_multiple_cv(classifiers, k, x, y, metric='roc_auc'):
    print('%s-fold cross validation:\n' % k)

    for clf in classifiers:
        scores = model_selection.cross_val_score(clf, x, y, 
                                                cv=k, scoring=metric)
        print("%s: %0.2f (+/- %0.2f) [%s]" 
            % (metric.upper(), scores.mean(), scores.std(), clf.__class__.__name__))

def test_multiple(classifiers, x, y, test_x, test_y, metric=roc_auc_score):
    for clf in classifiers:
        clf.fit(x, y)
        target = clf.predict(test_x)
        score = metric(target, test_y)
        print("%s: %0.2f - [%s]" % (metric.__name__.upper(), score, clf.__class__.__name__))

## Unormalized Data

We'll test the same classifiers on the raw data.

In [3]:
training_set = utils.load_training_set()

In [4]:
utils.percentage_missin(training_set)

,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,age,NumberOfTime6089DaysPastDueNotWorse,DebtRatio,NumberOfDependents,MonthlyIncome,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,NumberOfTime3059DaysPastDueNotWorse,NumberOfTimes90DaysLate
% of missing,0,0,0,0,0,2,19,0,0,0,0


In [5]:
# There are a percentage of missing values we'll fill with 0's
training_set = training_set.fillna(0)
training_set.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime3059DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime6089DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [6]:
utils.percentage_missin(training_set)

,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,age,NumberOfTime6089DaysPastDueNotWorse,DebtRatio,NumberOfDependents,MonthlyIncome,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,NumberOfTime3059DaysPastDueNotWorse,NumberOfTimes90DaysLate
% of missing,0,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn import preprocessing

y = training_set['SeriousDlqin2yrs'].as_matrix()
x = training_set.drop(['SeriousDlqin2yrs'], axis=1).as_matrix()
# Basic scaling
x = preprocessing.scale(x)

In [8]:
x_train, x_test, y_train, y_test = utils.split_dataset(x, y)

Dataset splitted from: 
	x=(150000, 10), y=(150000,)
To: 
Train Set
	x=(100500, 10), y=(100500,)
Test Set
	x=(49500, 10), y=(49500,)


## Algorithm Selection and Validation Selection

According to the Survey on the Competition Forum the Algorithms used were:

* [XGBooost](https://www.kaggle.com/c/GiveMeSomeCredit/discussion/31514)
* [AdaBoost, GradBoost, KNN, Logistic, and RandomForest](https://www.kaggle.com/c/GiveMeSomeCredit/discussion/23179)

The validation would be a simple Strat-K-FoldCV, and then use them on the test set, about 10% of the training set.

My selection of classifiers goes according to my Survey on the Forums, and my Proposal initial ideas.

> I've ordered them on this list according to the Proposal, with simpler models first, and other ideas.

In [9]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import (AdaBoostClassifier, RandomForestClassifier,
                              GradientBoostingClassifier, BaggingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
# Credits to: https://www.kaggle.com/c/GiveMeSomeCredit/discussion/31514
best_xgb = XGBClassifier(
    **{'min_child_weight': 10.0,
    'objective': 'binary:logistic',
    'max_depth': 5,
    'max_delta_step': 1.8,
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'gamma': 0.65,
    'n_estimators' : 391
    }
)
ann = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1)

classifiers = [GaussianNB(), 
               DecisionTreeClassifier(),
               LogisticRegression(),
               ann,
               # Ensembles
               AdaBoostClassifier(), 
               RandomForestClassifier(), 
               GradientBoostingClassifier(),
               BaggingClassifier(),
               best_xgb]

In [10]:
test_multiple_cv(classifiers, 10, x_train, y_train)

10-fold cross validation:

ROC_AUC: 0.71 (+/- 0.01) [GaussianNB]
ROC_AUC: 0.61 (+/- 0.01) [DecisionTreeClassifier]
ROC_AUC: 0.70 (+/- 0.01) [LogisticRegression]
ROC_AUC: 0.83 (+/- 0.01) [MLPClassifier]
ROC_AUC: 0.86 (+/- 0.01) [AdaBoostClassifier]
ROC_AUC: 0.78 (+/- 0.01) [RandomForestClassifier]
ROC_AUC: 0.86 (+/- 0.01) [GradientBoostingClassifier]
ROC_AUC: 0.78 (+/- 0.01) [BaggingClassifier]
ROC_AUC: 0.86 (+/- 0.01) [XGBClassifier]


In [11]:
test_multiple_cv(classifiers, 10, x_train, y_train, metric='accuracy')

10-fold cross validation:

ACCURACY: 0.93 (+/- 0.00) [GaussianNB]
ACCURACY: 0.90 (+/- 0.00) [DecisionTreeClassifier]
ACCURACY: 0.93 (+/- 0.00) [LogisticRegression]
ACCURACY: 0.93 (+/- 0.00) [MLPClassifier]
ACCURACY: 0.94 (+/- 0.00) [AdaBoostClassifier]
ACCURACY: 0.93 (+/- 0.00) [RandomForestClassifier]
ACCURACY: 0.94 (+/- 0.00) [GradientBoostingClassifier]
ACCURACY: 0.93 (+/- 0.00) [BaggingClassifier]
ACCURACY: 0.94 (+/- 0.00) [XGBClassifier]


**The accuracy is quite useless for this case/dataset**, even lower AUC [GaussianNB] get a good Accuracy.

We should test them in the test set to be sure.

In [12]:
test_multiple(classifiers, x_train, y_train, x_test, y_test)

ROC_AUC_SCORE: 0.67 - [GaussianNB]
ROC_AUC_SCORE: 0.60 - [DecisionTreeClassifier]
ROC_AUC_SCORE: 0.74 - [LogisticRegression]
ROC_AUC_SCORE: 0.74 - [MLPClassifier]
ROC_AUC_SCORE: 0.75 - [AdaBoostClassifier]
ROC_AUC_SCORE: 0.72 - [RandomForestClassifier]
ROC_AUC_SCORE: 0.77 - [GradientBoostingClassifier]
ROC_AUC_SCORE: 0.71 - [BaggingClassifier]
ROC_AUC_SCORE: 0.76 - [XGBClassifier]


> We have very ok values compared to the AUCs from the Competition on this simple test.

We'll need to find better feature sets before messing with the classifiers

## Features

These features are based on the [Winners Features](https://www.kaggle.com/c/GiveMeSomeCredit/discussion/1166#7269).

There are results around .8 > with these set of features, it's a good start.

Compared to the dataset feature set (10 features), there are 80 features, a lot more to work on.

In [10]:
features_set = utils.load_features_set()
features_set.head()

,SeriousDlqin2yrs,UnknownNumberOfDependents,UnknownMonthlyIncome,NoDependents,NoIncome,ZeroDebtRatio,UnknownIncomeDebtRatio,WeirdRevolvingUtilization,ZeroRevolvingUtilization,LogDebt,...,LogDebtPer90DaysLate,LogUnknownIncomeDebtRatio,LogUnknownIncomeDebtRatioPerPerson,LogUnknownIncomeDebtRatioPerLine,LogUnknownIncomeDebtRatioPerRealEstateLine,LogUnknownIncomeDebtRatioPerDelinquency,LogUnknownIncomeDebtRatioPer90DaysLate,LogNumberRealEstateLoansOrLines,LowAge,Logage
0,1,0,0,0,0,0,0,0.0,0,8.898802,...,8.898802,0.0,-1.098612,-2.639057,-1.945910,-1.098612,0.000000,1.791759,0,3.332205
1,0,0,0,0,0,0,0,0.0,0,5.758517,...,5.758517,0.0,-0.693147,-1.609438,0.000000,0.000000,0.000000,0.000000,0,3.135494
2,0,0,0,1,0,0,0,0.0,0,5.556499,...,4.863352,0.0,0.000000,-1.098612,0.000000,-1.098612,-0.693147,0.000000,0,3.044522
3,0,0,0,1,0,0,0,0.0,0,4.778821,...,4.778821,0.0,0.000000,-1.791759,0.000000,0.000000,0.000000,0.000000,0,2.564949
4,0,0,0,1,0,0,0,0.0,0,7.368324,...,7.368324,0.0,0.000000,-2.079442,-0.693147,-0.693147,0.000000,0.000000,0,3.465736


In [11]:
y = features_set['SeriousDlqin2yrs'].as_matrix()
x = features_set.drop(['SeriousDlqin2yrs'], axis=1).as_matrix()

In [12]:
x_train_f, x_test_f, y_train_f, y_test_f = utils.split_dataset(x, y)

Dataset splitted from: 
	x=(150000, 80), y=(150000,)
To: 
Train Set
	x=(100500, 80), y=(100500,)
Test Set
	x=(49500, 80), y=(49500,)


In [16]:
test_multiple_cv(classifiers, 10, x_train_f, y_train_f)

10-fold cross validation:

ROC_AUC: 0.83 (+/- 0.01) [GaussianNB]
ROC_AUC: 0.61 (+/- 0.00) [DecisionTreeClassifier]
ROC_AUC: 0.86 (+/- 0.01) [LogisticRegression]
ROC_AUC: 0.58 (+/- 0.02) [MLPClassifier]
ROC_AUC: 0.86 (+/- 0.01) [AdaBoostClassifier]
ROC_AUC: 0.78 (+/- 0.01) [RandomForestClassifier]
ROC_AUC: 0.86 (+/- 0.01) [GradientBoostingClassifier]
ROC_AUC: 0.78 (+/- 0.01) [BaggingClassifier]
ROC_AUC: 0.86 (+/- 0.01) [XGBClassifier]


In [17]:
test_multiple(classifiers, x_train_f, y_train_f, x_test_f, y_test_f)

ROC_AUC_SCORE: 0.67 - [GaussianNB]
ROC_AUC_SCORE: 0.60 - [DecisionTreeClassifier]
ROC_AUC_SCORE: 0.77 - [LogisticRegression]
ROC_AUC_SCORE: 0.47 - [MLPClassifier]
ROC_AUC_SCORE: 0.75 - [AdaBoostClassifier]
ROC_AUC_SCORE: 0.72 - [RandomForestClassifier]
ROC_AUC_SCORE: 0.77 - [GradientBoostingClassifier]
ROC_AUC_SCORE: 0.71 - [BaggingClassifier]
ROC_AUC_SCORE: 0.76 - [XGBClassifier]


> Seems like a good start, yet there were results with .8 AUC in the Test Set on the Competition.

** The MLP is too simple for this Feature set of 80 features**, it should be bigger

In [19]:
# It will take a long time to train
ann = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(40, 6), random_state=1)

test_multiple([ann], x_train_f, y_train_f, x_test_f, y_test_f)

ROC_AUC_SCORE: 0.50 - [MLPClassifier]


A simple MLP might not be that good, let's build a bigger one with Keras.

In [50]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import roc_auc_score
from keras_tqdm import TQDMCallback
from keras.callbacks import EarlyStopping

batch_size = 4096
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
progress = TQDMCallback(leave_outer=False)

model = Sequential()
model.add(Dense(160, activation='tanh', input_shape=(80,)))
model.add(Dense(80, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train_f, y_train_f,
          batch_size=batch_size,
          epochs=10, verbose=0,
          validation_data=(x_test_f, y_test_f),
          callbacks = [early, progress])

model.save('credit_scoring.h5')

y_pred = model.predict_classes(x_test_f)

print("%s: %0.2f - [%s]" % (roc_auc_score.__name__.upper(), roc_auc_score(y_pred, y_test_f), 'Keras MLP'))

Epoch: 0 - loss: 0.087, acc: 0.886:  86%|████████▌ | 86016/100500 [00:01<00:00, 27421.79it/s]
Epoch: 0 - loss: 0.084, acc: 0.892, val_loss: 0.065, val_acc: 0.931: 100%|██████████| 100500/100500 [00:01<00:00, 33953.05it/s]
Epoch: 1 - loss: 0.064, acc: 0.932:  94%|█████████▎| 94208/100500 [00:00<00:00, 219086.14it/s]
Epoch: 1 - loss: 0.064, acc: 0.931, val_loss: 0.062, val_acc: 0.933: 100%|██████████| 100500/100500 [00:00<00:00, 78759.26it/s]
Epoch: 2 - loss: 0.064, acc: 0.931:  94%|█████████▎| 94208/100500 [00:00<00:00, 175086.72it/s]
Epoch: 2 - loss: 0.064, acc: 0.931, val_loss: 0.070, val_acc: 0.916: 100%|██████████| 100500/100500 [00:00<00:00, 95347.80it/s]
Epoch: 3 - loss: 0.063, acc: 0.931:  82%|████████▏ | 81920/100500 [00:00<00:00, 176195.97it/s]
Epoch: 3 - loss: 0.063, acc: 0.931, val_loss: 0.064, val_acc: 0.930: 100%|██████████| 100500/100500 [00:00<00:00, 159572.28it/s]
Epoch: 4 - loss: 0.063, acc: 0.931:  82%|████████▏ | 81920/100500 [00:00<00:00, 249823.11it/s]
Epoch: 4 - lo

ROC_AUC_SCORE: 0.79 - [Keras MLP]


**Tahn in the middle layers, and sigmoid in the last layers shows us a good result.**

> ROC_AUC_SCORE: 0.79 - [Keras MLP] VS ROC_AUC_SCORE: 0.77 - [GradientBoostingClassifier] in our Test Set

With random Initializations I've managed between a 0.78 - 0.90 ROC_AUC_SCORE, so a NNET will be optimized with about 3 to 4 tanh hidden layers, and a sigmoid input/output layer.

Lets check the submission with this network model.

In [3]:
submission_set = utils.load_test_set()
submission_set.head()

,Id,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [4]:
submission_feature_set = utils.load_test_feature_set()
submission_feature_set = submission_feature_set.drop(['SeriousDlqin2yrs'], axis=1)
submission_feature_set.head()

,UnknownNumberOfDependents,UnknownMonthlyIncome,NoDependents,NoIncome,ZeroDebtRatio,UnknownIncomeDebtRatio,WeirdRevolvingUtilization,ZeroRevolvingUtilization,LogDebt,RevolvingLines,...,LogDebtPer90DaysLate,LogUnknownIncomeDebtRatio,LogUnknownIncomeDebtRatioPerPerson,LogUnknownIncomeDebtRatioPerLine,LogUnknownIncomeDebtRatioPerRealEstateLine,LogUnknownIncomeDebtRatioPerDelinquency,LogUnknownIncomeDebtRatioPer90DaysLate,LogNumberRealEstateLoansOrLines,LowAge,Logage
0,0,0,1,0,0,0,0.0,0,6.919508,4,...,6.919508,0.0,0.000000,-1.609438,0.000000,0.000000,0.0,0.000000,0,3.258097
1,0,0,0,0,0,0,0.0,0,8.480420,11,...,8.480420,0.0,-1.098612,-2.772589,-1.609438,0.000000,0.0,1.386294,0,3.688879
2,0,0,0,0,0,0,0.0,0,8.159178,11,...,8.159178,0.0,-1.098612,-2.564949,-0.693147,0.000000,0.0,0.000000,0,3.737670
3,0,0,1,0,0,0,0.0,0,7.993983,5,...,7.993983,0.0,0.000000,-2.079442,-1.098612,-0.693147,0.0,0.693147,0,3.044522
4,0,0,0,0,0,0,0.0,0,4.343547,4,...,4.343547,0.0,-0.693147,-1.609438,0.000000,0.000000,0.0,0.000000,0,2.302585


In [9]:
from keras.models import load_model
# load the keras model
model = load_model('credit_scoring.h5')

x = submission_feature_set.as_matrix()

y_predicts = model.predict_classes(x)
y_probs = model.predict(x)

submission_set.SeriousDlqin2yrs = y_predicts
submission_set['Probability'] = y_probs

submission_set.head()

,Id,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,Probability
0,1,0,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0,0.078274
1,2,0,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0,0.121120
2,3,0,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0,0.048864
3,4,0,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0,0.105179
4,5,0,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0,0.126238


In [14]:
submission_set.count()
# We expect the solution file to have 101503 prediction rows.

Id                                      101503
SeriousDlqin2yrs                        101503
RevolvingUtilizationOfUnsecuredLines    101503
age                                     101503
NumberOfTime30-59DaysPastDueNotWorse    101503
DebtRatio                               101503
MonthlyIncome                            81400
NumberOfOpenCreditLinesAndLoans         101503
NumberOfTimes90DaysLate                 101503
NumberRealEstateLoansOrLines            101503
NumberOfTime60-89DaysPastDueNotWorse    101503
NumberOfDependents                       98877
Probability                             101503
dtype: int64

** Lets count the predictions of each class, check if it's ok (not just full one class or anything) **

In [10]:
submission_set.query('SeriousDlqin2yrs == 0').count()

Id                                      101467
SeriousDlqin2yrs                        101467
RevolvingUtilizationOfUnsecuredLines    101467
age                                     101467
NumberOfTime30-59DaysPastDueNotWorse    101467
DebtRatio                               101467
MonthlyIncome                            81369
NumberOfOpenCreditLinesAndLoans         101467
NumberOfTimes90DaysLate                 101467
NumberRealEstateLoansOrLines            101467
NumberOfTime60-89DaysPastDueNotWorse    101467
NumberOfDependents                       98841
Probability                             101467
dtype: int64

In [11]:
submission_set.query('SeriousDlqin2yrs == 1').count()

Id                                      36
SeriousDlqin2yrs                        36
RevolvingUtilizationOfUnsecuredLines    36
age                                     36
NumberOfTime30-59DaysPastDueNotWorse    36
DebtRatio                               36
MonthlyIncome                           31
NumberOfOpenCreditLinesAndLoans         36
NumberOfTimes90DaysLate                 36
NumberRealEstateLoansOrLines            36
NumberOfTime60-89DaysPastDueNotWorse    36
NumberOfDependents                      36
Probability                             36
dtype: int64

In [15]:
# Submission must be prob and id:
import pandas as pd
submission_set.to_csv(utils.os.path.join(utils.DIR, 'dataset', 'submission.csv'),
                      index=False, 
                      columns=['Id', 'Probability'])

** First submission was bad **:
![](images/first_submission.png)

Training the classifier with this set of data might not be ok, we need to split it better.
And we also need to compute the ROC_AUC_SCORE at the end of each epoch on the validation set.
We might do that using a Callback function.

## Checkpoint

Second try, using a smaller dataset split only

In [1]:
def build_submission_keras(model):
    submission_feature_set = utils.load_test_feature_set()
    submission_feature_set = submission_feature_set.drop(['SeriousDlqin2yrs'], axis=1)
    x = submission_feature_set.as_matrix()

    y_predicts = model.predict_classes(x)
    y_probs = model.predict(x)
    
    submission_set = utils.load_test_set()
    submission_set .SeriousDlqin2yrs = y_predicts
    submission_set ['Probability'] = y_probs
    
    return submission_set

In [2]:
import utils
features_set = utils.load_features_set()
y = features_set['SeriousDlqin2yrs'].as_matrix()
x = features_set.drop(['SeriousDlqin2yrs'], axis=1).as_matrix()
x_train_f, x_test_f, y_train_f, y_test_f = utils.split_dataset(x, y, portion=0.10)

/media/rodsnjr/Files/miniconda3/envs/udacity-ml/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Dataset splitted from: 
	x=(150000, 80), y=(150000,)
To: 
Train Set
	x=(135000, 80), y=(135000,)
Test Set
	x=(15000, 80), y=(15000,)


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import roc_auc_score
from keras_tqdm import TQDMCallback
from keras.callbacks import EarlyStopping, LambdaCallback

def roc_callback(epochs, logs):
    try:
        print 'EPOCH %s ROC_AUC_SCORE %s' % (epochs, roc_auc_score(model.predict_classes(x_test_f), y_test_f))
    except ValueError:
        print 'ROC_AUC_SCORE is 0, only one class present in the predictions'

batch_size = 4096
early = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')
roc_scoring = LambdaCallback(on_epoch_end=lambda epochs, logs: roc_callback(epochs, logs))
progress = TQDMCallback(leave_outer=False)

model = Sequential()
model.add(Dense(80, activation='tanh', input_shape=(80,)))
model.add(Dense(60, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train_f, y_train_f,
          batch_size=batch_size,
          epochs=30, verbose=0,
          validation_data=(x_test_f, y_test_f),
          callbacks = [early, progress, roc_scoring])

model.save('credit_scoring_2.h5')

y_pred = model.predict_classes(x_test_f)

print("%s: %0.2f - [%s]" % (roc_auc_score.__name__.upper(), roc_auc_score(y_pred, y_test_f), 'Keras MLP'))

Epoch: 0 - loss: 0.108, acc: 0.856:  88%|████████▊ | 118784/135000 [00:00<00:00, 74834.62it/s]
Epoch: 0 - loss: 0.104, acc: 0.863, val_loss: 0.067, val_acc: 0.926: 100%|██████████| 135000/135000 [00:00<00:00, 88056.03it/s]
Epoch: 1 - loss: 0.068, acc: 0.928:  21%|██        | 28672/135000 [00:00<00:00, 281592.36it/s]

EPOCH 0 ROC_AUC_SCORE 0.538942180846



Epoch: 1 - loss: 0.064, acc: 0.932:  88%|████████▊ | 118784/135000 [00:00<00:00, 208496.03it/s]
Epoch: 1 - loss: 0.065, acc: 0.932, val_loss: 0.060, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 183562.47it/s]
Epoch: 2 - loss: 0.063, acc: 0.933:  21%|██        | 28672/135000 [00:00<00:00, 248266.06it/s]

EPOCH 1 ROC_AUC_SCORE 0.593041099546



Epoch: 2 - loss: 0.063, acc: 0.932:  94%|█████████▍| 126976/135000 [00:00<00:00, 216458.57it/s]
Epoch: 2 - loss: 0.063, acc: 0.932, val_loss: 0.060, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 201975.52it/s]
Epoch: 3 - loss: 0.063, acc: 0.932:  18%|█▊        | 24576/135000 [00:00<00:00, 232998.45it/s]

...


Epoch: 28 - loss: 0.053, acc: 0.934:  97%|█████████▋| 131072/135000 [00:00<00:00, 203365.45it/s]
Epoch: 28 - loss: 0.053, acc: 0.934, val_loss: 0.051, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 126677.91it/s]
Epoch: 29 - loss: 0.053, acc: 0.934:  18%|█▊        | 24576/135000 [00:00<00:00, 214328.64it/s]

EPOCH 28 ROC_AUC_SCORE 0.705851754151



Epoch: 29 - loss: 0.052, acc: 0.935:  79%|███████▉  | 106496/135000 [00:00<00:00, 238886.49it/s]
Epoch: 29 - loss: 0.052, acc: 0.934, val_loss: 0.053, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 243907.80it/s]
                                                         

EPOCH 29 ROC_AUC_SCORE 0.751346157201


ROC_AUC_SCORE: 0.75 - [Keras MLP]


In [9]:
import pandas as pd

submission_set = build_submission_keras(model)

submission_set.to_csv(utils.os.path.join(utils.DIR, 'dataset', 'submission.csv'),
                      index=False, 
                      columns=['Id', 'Probability'])

** Second submission was ok **:

![](images/first_submission.png)

Remember, the competition avg scores from the top 350 +- was around 0.86, so we can still improve our model.

The number of the hidden layers is good, and the number of hidden units is also good.
The improvements made were on the number of training epochs, and the optimizer to adam.

We also had epochs were the ROC_AUC was bigger than 0.80, so we'll checkpoint those.

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import roc_auc_score
from keras_tqdm import TQDMCallback
from keras.callbacks import EarlyStopping, LambdaCallback

def roc_callback(epochs, logs):
    try:
        score = roc_auc_score(model.predict_classes(x_test_f), y_test_f)
        
        if score > 0.80:
            model.save('credit_scoring_3_spc.h5')
        
        print 'EPOCH %s ROC_AUC_SCORE %s' % (epochs, score)
    except ValueError:
        print 'ROC_AUC_SCORE is 0, only one class present in the predictions'

batch_size = 4096
early = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')
roc_scoring = LambdaCallback(on_epoch_end=lambda epochs, logs: roc_callback(epochs, logs))
progress = TQDMCallback(leave_outer=False)

model = Sequential()
model.add(Dense(80, activation='tanh', input_shape=(80,)))
model.add(Dense(70, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(50, activation='tanh'))
model.add(Dense(40, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train_f, y_train_f,
          batch_size=batch_size,
          epochs=50, verbose=0,
          validation_data=(x_test_f, y_test_f),
          callbacks = [early, progress, roc_scoring])

model.save('credit_scoring_3.h5')

y_pred = model.predict_classes(x_test_f)

print("%s: %0.2f - [%s]" % (roc_auc_score.__name__.upper(), roc_auc_score(y_pred, y_test_f), 'Keras MLP'))

Epoch: 0 - loss: 0.366, acc: 0.872:  91%|█████████ | 122880/135000 [00:01<00:00, 81136.04it/s]
Epoch: 0 - loss: 0.360, acc: 0.875, val_loss: 0.277, val_acc: 0.913: 100%|██████████| 135000/135000 [00:01<00:00, 79844.32it/s]
Epoch: 1 - loss: 0.281, acc: 0.909:  18%|█▊        | 24576/135000 [00:00<00:00, 224489.82it/s]

EPOCH 0 ROC_AUC_SCORE 0.535307579383



Epoch: 1 - loss: 0.257, acc: 0.926:  91%|█████████ | 122880/135000 [00:00<00:00, 182205.62it/s]
Epoch: 1 - loss: 0.256, acc: 0.927, val_loss: 0.238, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 176682.22it/s]
Epoch: 2 - loss: 0.243, acc: 0.934:  18%|█▊        | 24576/135000 [00:00<00:00, 220600.72it/s]

EPOCH 1 ROC_AUC_SCORE 0.593016275347



Epoch: 2 - loss: 0.246, acc: 0.933:  91%|█████████ | 122880/135000 [00:00<00:00, 173521.97it/s]
Epoch: 2 - loss: 0.247, acc: 0.932, val_loss: 0.238, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 148404.76it/s]
Epoch: 3 - loss: 0.247, acc: 0.932:  15%|█▌        | 20480/135000 [00:00<00:00, 182149.52it/s]

...


Epoch: 48 - loss: 0.184, acc: 0.935:  94%|█████████▍| 126976/135000 [00:00<00:00, 157827.72it/s]
Epoch: 48 - loss: 0.184, acc: 0.935, val_loss: 0.180, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 129300.19it/s]
Epoch: 49 - loss: 0.185, acc: 0.935:  12%|█▏        | 16384/135000 [00:00<00:00, 156981.59it/s]

EPOCH 48 ROC_AUC_SCORE 0.721822147694



Epoch: 49 - loss: 0.188, acc: 0.935:  94%|█████████▍| 126976/135000 [00:00<00:00, 161813.65it/s]
Epoch: 49 - loss: 0.187, acc: 0.935, val_loss: 0.186, val_acc: 0.936: 100%|██████████| 135000/135000 [00:00<00:00, 135067.71it/s]
                                                         

EPOCH 49 ROC_AUC_SCORE 0.748504987315


ROC_AUC_SCORE: 0.75 - [Keras MLP]


In [13]:
# We have both models saved, so we'll load and submit both.
from keras.models import load_model
import pandas as pd

fully_trained = load_model('credit_scoring_3.h5')

submission_set = build_submission_keras(fully_trained)
submission_set.to_csv(utils.os.path.join(utils.DIR, 'dataset', 'submission_last.csv'),
                      index=False, 
                      columns=['Id', 'Probability'])

special_model = load_model('credit_scoring_3_spc.h5')

submission_set = build_submission_keras(special_model)
submission_set.to_csv(utils.os.path.join(utils.DIR, 'dataset', 'submission_last_spc.csv'),
                      index=False, 
                      columns=['Id', 'Probability'])

Both results:

** The fully trained model submission result **:

![](images/best_score_ann.png)

** The trained model with roc_auc > 0.80 in the test set submission result **:

![](images/special_submission.png)

It's hard to evaluate the network perfomance since the loss functions are calculated in a way that it's not optimized for this kind of problem.

And it's also hard to evaluate the accuracy without the roc_auc, and without a proper validation and test set.

The BOOSTING models were a lot better.

But I've decided to check a NN based approach, since deep neural networks are now a trend.

## Checkpoint

Let's try the boosting classifier

In [8]:
import utils
features_set = utils.load_features_set()
y = features_set['SeriousDlqin2yrs'].as_matrix()
x = features_set.drop(['SeriousDlqin2yrs'], axis=1).as_matrix()
x_train_f, x_test_f, y_train_f, y_test_f = utils.split_dataset(x, y, portion=0.20)

Dataset splitted from: 
	x=(150000, 80), y=(150000,)
To: 
Train Set
	x=(120000, 80), y=(120000,)
Test Set
	x=(30000, 80), y=(30000,)


In [9]:
def build_submission_skl(classifier):
    submission_feature_set = utils.load_test_feature_set()
    submission_feature_set = submission_feature_set.drop(['SeriousDlqin2yrs'], axis=1)
    x = submission_feature_set.as_matrix()

    y_predicts = classifier.predict(x)
    y_probs = classifier.predict_proba(x)
    
    submission_set = utils.load_test_set()
    submission_set .SeriousDlqin2yrs = y_predicts
    submission_set ['Probability'] = y_probs[:,:1]
    
    return submission_set

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

gb_clf = GradientBoostingClassifier()

gb_clf.fit(x_train_f, y_train_f)

#y_predicts = gb_clf.predict(x[:10])
#y_probs = gb_clf.predict_proba(x[:10])

submission_set = build_submission_skl(gb_clf)
submission_set.to_csv(utils.os.path.join(utils.DIR, 'dataset', 'submission_gb.csv'),
                      index=False, 
                      columns=['Id', 'Probability'])

** The simple GradientBoostingClassifier without optimization got .86 **:

![](images/gb_submission.png)

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import make_scorer, roc_auc_score

clf = GradientBoostingClassifier()

cv_sets = ShuffleSplit(x_train_f.shape[0], n_iter = 10, 
                       test_size = 0.1, train_size=None, 
                       random_state = 0)

parameters = {
    'min_samples_split': [2, 3 ],
    'n_estimators' : [200, 300, 400],
    'max_features': ['log2', 'sqrt']
}

scorer = make_scorer(roc_auc_score)
             
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=scorer, cv=cv_sets, n_jobs=2, verbose=6)

grid_obj = grid_obj.fit(x_train_f, y_train_f)

clf = grid_obj.best_estimator_

y_targets = clf.predict(x_test_f)

score = roc_auc_score(y_targets, y_test_f)

print("%s: %0.8f - [%s]" % ('ROC_AUC_SCORE', score, gb_clf.__class__.__name__))

submission_set = build_submission_skl(clf)
submission_set.to_csv(utils.os.path.join(utils.DIR, 'dataset', 'optimized_gb_pc.csv'),
                      index=False, 
                      columns=['Id', 'Probability'])

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] max_features=log2, min_samples_split=2, n_estimators=200 ........
[CV] max_features=log2, min_samples_split=2, n_estimators=200 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=200, score=0.599307 -  14.9s
[CV] max_features=log2, min_samples_split=2, n_estimators=200 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=200, score=0.593204 -  15.0s
[CV] max_features=log2, min_samples_split=2, n_estimators=200 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=200, score=0.583706 -  15.2s
[CV] max_features=log2, min_samples_split=2, n_estimators=200 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=200, score=0.601291 -  15.2s
[CV] max_features=log2, min_samples_split=2, n_estimators=200 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=200, score=0.593238 -  14.6s
[CV] max_features=log2, min_samples_split=2, n_estimators=200 ........


[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:  1.2min


[CV]  max_features=log2, min_samples_split=2, n_estimators=200, score=0.588104 -  15.0s
[CV] max_features=log2, min_samples_split=2, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=300, score=0.592137 -  20.9s
[CV] max_features=log2, min_samples_split=2, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=300, score=0.596203 -  21.1s
[CV] max_features=log2, min_samples_split=2, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=300, score=0.588605 -  21.2s
[CV] max_features=log2, min_samples_split=2, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=300, score=0.591959 -  21.4s
[CV] max_features=log2, min_samples_split=2, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=2, n_estimators=300, score=0.594312 -  21.2s
[CV] max_features=log2, min_samples_split=2, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=2, 

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  7.6min


[CV]  max_features=log2, min_samples_split=3, n_estimators=300, score=0.595231 -  21.0s
[CV] max_features=log2, min_samples_split=3, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=3, n_estimators=300, score=0.595542 -  21.7s
[CV] max_features=log2, min_samples_split=3, n_estimators=300 ........
[CV]  max_features=log2, min_samples_split=3, n_estimators=300, score=0.595926 -  21.3s
[CV] max_features=log2, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=log2, min_samples_split=3, n_estimators=300, score=0.590119 -  21.7s
[CV] max_features=log2, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=log2, min_samples_split=3, n_estimators=400, score=0.598542 -  28.2s
[CV] max_features=log2, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=log2, min_samples_split=3, n_estimators=400, score=0.596850 -  28.1s
[CV] max_features=log2, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=log2, min_samples_split=3, 

[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed: 20.8min


[CV]  max_features=sqrt, min_samples_split=3, n_estimators=300, score=0.599943 -  25.8s
[CV] max_features=sqrt, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=sqrt, min_samples_split=3, n_estimators=400, score=0.595130 -  34.0s
[CV] max_features=sqrt, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=sqrt, min_samples_split=3, n_estimators=400, score=0.594360 -  34.3s
[CV] max_features=sqrt, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=sqrt, min_samples_split=3, n_estimators=400, score=0.584196 -  34.2s
[CV] max_features=sqrt, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=sqrt, min_samples_split=3, n_estimators=400, score=0.601652 -  34.2s
[CV] max_features=sqrt, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=sqrt, min_samples_split=3, n_estimators=400, score=0.596396 -  33.3s
[CV] max_features=sqrt, min_samples_split=3, n_estimators=400 ........
[CV]  max_features=sqrt, min_samples_split=3, 

[Parallel(n_jobs=2)]: Done 120 out of 120 | elapsed: 23.7min finished


ROC_AUC_SCORE: 0.76844380 - [GradientBoostingClassifier]


> The tryin to tune the model gave:

![](images/bad_optimized_gb.png)

In [2]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score

import utils

def test_clf(clf, x_train, y_train, x_test, y_test):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("%s: %0.8f - [%s]" % (roc_auc_score.__name__.upper(), roc_auc_score(y_pred, y_test), clf.__class__.__name__))
    
    return clf

def build_submission_skl(classifier):
    submission_feature_set = utils.load_test_feature_set()
    submission_feature_set = submission_feature_set.drop(['SeriousDlqin2yrs'], axis=1)
    x = submission_feature_set.as_matrix()

    y_predicts = classifier.predict(x)
    y_probs = classifier.predict_proba(x)
    
    submission_set = utils.load_test_set()
    submission_set .SeriousDlqin2yrs = y_predicts
    submission_set ['Probability'] = y_probs[:,:1]
    
    return submission_set

features_set = utils.load_features_set()
y = features_set['SeriousDlqin2yrs'].as_matrix()
x = features_set.drop(['SeriousDlqin2yrs'], axis=1).as_matrix()

x_train_f, x_test_f, y_train_f, y_test_f = utils.split_dataset(x, y, portion=0.20)

/media/rodsnjr/Files/miniconda3/envs/udacity-ml/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Dataset splitted from: 
	x=(150000, 80), y=(150000,)
To: 
Train Set
	x=(120000, 80), y=(120000,)
Test Set
	x=(30000, 80), y=(30000,)


In [3]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=391, 
                                 max_depth=4, 
                                 subsample=0.8,
                                 min_samples_split=3,
                                 min_weight_fraction_leaf=0.2,
                                 min_samples_leaf=2, verbose=1)

gb_clf = test_clf(clf, x_train_f, y_train_f, x_test_f, y_test_f)

submission_set = build_submission_skl(gb_clf)
submission_set.to_csv(utils.os.path.join(utils.DIR, 'dataset', 'fs_submission_gb.csv'),
                      index=False, 
                      columns=['Id', 'Probability'])

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.4715           0.0198            2.45m
         2           0.4586           0.0143            2.46m
         3           0.4473           0.0105            2.44m
         4           0.4383           0.0082            2.42m
         5           0.4351           0.0062            2.41m
         6           0.4285           0.0052            2.41m
         7           0.4227           0.0050            2.40m
         8           0.4226           0.0038            2.39m
         9           0.4155           0.0032            2.38m
        10           0.4132           0.0036            2.37m
        20           0.3949           0.0010            2.24m
        30           0.3874           0.0006            2.19m
        40           0.3830           0.0003            2.17m
        50           0.3798           0.0002            2.17m
        60           0.3766           0.0001            2.11m
       

By changing a few parameters manually and submitting I've managed this

![](images/better_n_estimators.png)